In [45]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import spacy
import sys
sys.path.append('../../../../utils')
sys.path.append('..')
import gezi
from gezi import tqdm
from src.eval import calc_f1
from src.visualize import *
from src.config import *
from src.util import *
pd.set_option('display.float_format', lambda x: '%.2f' % x)
gezi.set_pandas_widder()

In [2]:
texts_csv = '../input/feedback-prize-2021/texts.fea'
df_texts = pd.read_feather(texts_csv)
df_texts.head()

,index,id,text
0,0,4B22D00F9BD7,Students of the United States of America have ...
1,1,1F8DDCFCA766,"Have you ever awoken to the arrogant, angry sh..."
2,2,ED27B5020694,"The Battle for Control\n\nIn the near future, ..."
3,3,B1AF3AD74555,In the article the author talks about machines...
4,4,F0B1C389A815,Have you ever needed some dire advice for some...


In [3]:
with open('../input/feedback-prize-2021/train_corpus.txt', 'w') as f:
  for text in tqdm(df_texts.text.values, total=len(df_texts)):
    print(text.strip(), file=f)

  0%|          | 0/15594 [00:00<?, ?it/s]

In [4]:
df_texts.text.values[0]

"Students of the United States of America have you ever wondered what it would be like to do school work from home rather than actually go to the school and learn there. Students in college have had the option to take online courses instead of actually attending a college class directly for a while now. Now some high schools\xa0are beginning to offer up a similar program to students as of recently. Students it is a lot better to do school work from home because students would feel better in a familiar environment therefore allowing them to work more efficiently, a school could also save much more money on transportation costs allowing a school to have a bigger budget and finally the recent COVID-19 case has scared much of America today, so by allowing students to work from home there is a substantial decrease in the case a disease like this spreads quicker.\n\nAlthough some may argue against this by saying that if students don't have access to computers at home how would this work. The

In [5]:
df_texts.text.values[1]

"Have you ever awoken to the arrogant, angry shriek of your alarm clock on a Monday morning? Maybe you have even experienced the arctic splash of freezing cold water falling over your back as your mom struggled to pry you from a deep sleep. It is apparent that waking up for school each morning is one of the most dreadful moments a student will face. A potential solution to this issue is the introduction of online classes and video conferences. Through this much more compatible, modern way of learning, students would be less restricted and undoubtedly happier. By establishing the opportunity to attend classes from home, students will be able to spend significantly more time with their loved ones, obtain important skills including responsibility and time management, and even be able to improve their sleep schedules.\n\nRecently, it has become difficult for students to spend enough time at home with their families, or even to establish a strong social life. Between countless extracurricul

In [6]:
df_texts.text.values[2]

"The Battle for Control\n\nIn the near future, driveless cars will become an inevitable reality. Their a supposed solution for most traffic problems the world posseses today. No need to worry about drunk and reckless drivers anymore, the answer is finally here! Well, dive a little deeper and you'll figure out that driveless cars will create just as many problems as they would, in theory, fix. The idea of a world with driveless cars as the main priority just runs shivers down my spine...\n\nThe main problem with driveless cars is a very nightmarish one. What if the car malfunctions, and goes completely berzerk on the road? Also, who would take the blame for the accident? The manufacturers or the driver would be a re-occuring question, and as stated in the article, this would cause thousands of legal battles every week. It would be truly heartbreaking to go to someone's funeral that you were very close with, and to always know that their death wasn't even their fault.\n\nAnother issue dr

In [7]:
df_texts.text.values[-10]

'Throughout the duration of time, education has been a prime characteristic of the structure of society. As time progressed, the education system has changed such that it is more ideal for people in society. For example, in the early 1900s, the idea of summer breaks was introduced in the American education system. A summer break can be defined as a time in which students and teachers in the public education system get a 2-3 month grace period of not attending school. During this time, there is a common trend for teachers assign homework for students to complete for the next annual school year. Although some may argue that these summer assignments should be designed by students, summer assignments must be teacher designed because the teacher knows the content better than a student, it retains the student\'s memory efficiently, and students are prepared for the next academic year.\n\nSummer assignments must be teacher designed so that the memory of the student is exercised and retained. 

In [8]:
df_texts.text.values[-100]

"Do you enjoy staying after school?,what do you do when you get home from school?\n\nAre you busy when you get home?\n\nWell if students are required to stay after school for an activity,i think they should at least be rewarded for there hard work and extra time they put in. One reason i disagree with the principal is beacause what if most students all ready have plans after school and its mandatory to be there,some students are super tired/stressed out when they get home cause they have so much work to do for school or its been a long day ,and my last reason is because teachers are going to have to grade more work and most kids dont even stay after school to get work done they only stay because there friends are there to goof around with them.\n\nMy first reason to why students should not stay after school for a activity is because some students allready have plans after school and it's mandatory to be there. Once my little cousin's grandma got the flue and she was sent to the hospita

In [9]:
df_texts.text.values[0].count('\n')

10

In [10]:
df_texts.text.values[0].count('\xa0')

2

In [11]:
df_texts.text.values[0].count('\t')

0

In [12]:
df_texts.text.values[0].count('\r')

0

In [13]:
all = ' '.join(df_texts.text.values)

In [14]:
all.count('\n')

142225

In [15]:
all.count('\xa0')

65227

In [16]:
x = 'abc\nde \xa0b\nc\n\nx'

In [17]:
x.split()

['abc', 'de', 'b', 'c', 'x']

In [18]:
x.replace('\n', ' [BR] ').replace('\xa0', ' [XA0] ').split()

['abc', '[BR]', 'de', '[XA0]', 'b', '[BR]', 'c', '[BR]', '[BR]', 'x']

In [19]:
!ls ../input/asap-aes

Essay_Set_Descriptions	    training_set_rel3.xlsx
Essay_Set_Descriptions.zip  valid_sample_submission_1_column.csv
Training_Materials	    valid_sample_submission_1_column_no_header.csv
Training_Materials.zip	    valid_sample_submission_2_column.csv
asap-aes.zip		    valid_sample_submission_5_column.csv
test_set.tsv		    valid_set.tsv
training_set_rel3.tsv	    valid_set.xls
training_set_rel3.xls	    valid_set.xlsx


In [20]:
root = '../input/asap-aes'
files = [f'{root}/test_set.tsv', f'{root}/valid_set.tsv', f'{root}/training_set_rel3.tsv']

In [21]:
test = pd.read_csv(files[0], sep='\t', encoding='ISO-8859-1')

In [22]:
valid = pd.read_csv(files[1], sep='\t', encoding='ISO-8859-1')

In [23]:
train = pd.read_csv(files[2], sep='\t', encoding='ISO-8859-1')

In [24]:
train_text = ' '.join(train.essay.values)

In [25]:
train_text.count('\xa0')

0

In [29]:
train.iloc[0].essay

"Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenager is always on the phone with friends! Do you ever time to chat with your friends or buisness partner about things. Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: @ORGANIZATION1, @ORGANIZATION2, @CAPS1, facebook, myspace ect. Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. How did you learn about other countrys/states outside of yours? Well I have by computer/internet, it's a new way to learn about what going on in our time! You might think your child spends a lot of time on the computer, but ask them so question about the econom

In [38]:
all = pd.concat([train[['essay_id', 'essay']], valid[['essay_id', 'essay']], test[['essay_id', 'essay']]])
all['essay_id'] = all['essay_id'].astype(str)

In [41]:
all['num_words'] = all['essay'].apply(lambda x: len(x.split()))

In [53]:
all[all.num_words > 100].to_csv('../input/feedback-prize-2021/ext.csv', index=False)

In [42]:
all.describe()

,num_words
count,21448.00
mean,223.52
std,176.88
min,2.00
25%,99.00
50%,164.00
75%,307.00
max,1064.00


In [52]:
all[all.num_words > 100]

essay_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        